# MSDS3436 Final Project
<h3>Face Mask Detection using Sage Maker</h3>

2022WI_MSDS_436-DL_SEC55 Analytics Systems Engineering

Mark Stockwell | Wesley Ng

# Introduction
This project trains a ML model and deploys it to AWS SageMaker. The model is an image classification algorithm that does a binary classification of facial images to determine if they are wearing a mask or not. This file allows the user to upload a zip file of facial images and have them classified by the model.

# Setup

### Directory Structure
This project has base files in the root directory (this file), utility files in the /scripts directory, and data in the /data folder:
```
ROOT<br>
├───data<br>
│   ├───test<br>
│   │   ├───mask<br>
│   │   └───no_mask<br>
│   └───train<br>
│       ├───no_mask<br>
│       └───mask<br>
├───scripts<br>
├───model<br>
└───output<br>
```
### Step 1: Gather Training Data

The training data comes from a [Kaggle dataset](https://www.kaggle.com/pranavsingaraju/facemask-detection-dataset-20000-images ) of 20K split evenly between images of faces with masks and without. The data can be downloaded locally and replicated to the S3 bucket. You will need an AWS account and API key.  This file creates the necessary folders and copies to S3:

[/scripts/ProcessKaggleDataset.ipynb](/scripts/ProcessKaggleDataset.ipynb)
(links not active in Colab)

### Step 2: Train Model
The model is created and trained using the [/scripts/MaskDetectionModel.ipynb](/scripts/MaskDetectionModel.ipynb) file and creates a Sequence model with several layers:
```
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 50)                1638450   
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
=================================================================
Total params: 16,353,189
Trainable params: 1,638,501
Non-trainable params: 14,714,688
_________________________________________________________________
```

### Step 3: Deploy Model
The model is deployed using the [/scripts/DeploySagemakerModel.ipynb](/scripts/DeploySagemakerModel.ipynb) file. 

## Usage


## Upload Files

Upload a zip file from Google Drive with a set of images of people, some with face masks and some without. The file will be unzipped into the /input directory.

In [ ]:
!pip3 -q install --upgrade google-api-python-client 
from google.colab import files
import zipfile
import io
uploaded = files.upload()
with zipfile.ZipFile(list(uploaded.keys())[0], 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall(path='./input')

# all ok? this command should work
! ls -l -r

Saving Module5-Exercises.zip to Module5-Exercises.zip
total 10700
drwxr-xr-x 1 root root    4096 Mar  9 14:48 sample_data
-rw-r--r-- 1 root root  645736 Mar 11 05:06 Module5-Exercises.zip
-rw-r--r-- 1 root root 3087537 Mar 11 05:03 Module2-Exercises.zip
-rw-r--r-- 1 root root 7209426 Mar 11 04:58 Module1-Exercises.zip
drwxr-xr-x 5 root root    4096 Mar 11 05:06 input


## Process Data

**AWS Setup**

AWS authentication requires a ".aws" directory in your home directory (i.e. "~"). This directory will contain two files (see below). If using Colab you will need to upload the two files with no extension, "credentials" and "config". If running locally then make sure those two files in the .aws directory.

The credentials file has your API key, config file has preferred region

Put the following in each, change values to yours:
<hr>
credentials

```
[default]
aws_access_key_id = AKIAXN6R7JXYxxxxxxxx 
aws_secret_access_key = 1prI4W8J254zR5/ishEilmqskaZLOnazxxxxxxxx
```
<hr>
config

```
[default]
region = us-east-1
```

In [ ]:
!apt-get --qq -y update
# note versions compatible with Python 3.7 that comes with Colab
!pip3 -q install awscli==1.18
!pip3 -q install boto3==1.18
!pip3 -q install sagemaker==2.6

import boto3
from google.colab import files
files.upload()
! mkdir ~/.aws
! cp credentials ~/.aws/
! cp config ~/.aws/
! chmod 600 ~/.aws/credentials
! chmod 600 ~/.aws/config

# all ok? this command should work
comprehend = boto3.client(service_name='comprehend', region_name="us-east-1")
text = "Machine learning will automate jobs that most people thought could only be done by people." #~Dave Waters
print("text to perform sentiment analysis: \n",text)
comprehend.detect_sentiment(Text=text, LanguageCode='en')

### Upload input files to S3
The input files will be processed in batch mode by SageMaker.

In [ ]:
from re import sub
import boto3
import os
 
def upload_files(path,bucket):
    session = boto3.Session()
    s3 = session.resource('s3')
    bucket = s3.Bucket(bucket)
    filecount = 0
    for subdir, dirs, files in os.walk(path):
      print('uploading dir',subdir)
      for file in files:
        full_path = os.path.join(subdir, file)
        with open(full_path, 'rb') as data:
          # force into "input" folder
          bucket.put_object(Key='input/'+ full_path[len(path)+1:], Body=data)
        filecount +=1
      print('  file count:',filecount)

# call function
upload_files('./input','511004593648-msds436')

## Connect to Sagemaker and process the images

In [ ]:
%%time
import boto3
import re
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

role = get_execution_role()

bucket = sagemaker.Session().default_bucket()

training_image = get_image_uri(boto3.Session().region_name, "image-classification")

## Display Results

In [ ]:
# Code will display file names and thumbnails with labels and download link